<a href="https://colab.research.google.com/github/dkovah/testing_stable_diffusion/blob/main/testing_stable_diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Requisitos previos

In [1]:
%%capture
#@title 1- Instalar dependencias { display-mode: "form" }
!pip install diffusers==0.3.0
!pip install transformers scipy ftfy
!pip install "ipywidgets>=7,<8"
!git clone https://github.com/sberbank-ai/Real-ESRGAN
%cd Real-ESRGAN
!pip install -r requirements.txt
!gdown https://drive.google.com/uc?id=1pG2S3sYvSaO0V0B8QPOl1RapPHpUGOaV -O weights/RealESRGAN_x2.pth
!gdown https://drive.google.com/uc?id=1SGHdZAln4en65_NQeQY9UjchtkEF9f5F -O weights/RealESRGAN_x4.pth
!gdown https://drive.google.com/uc?id=1mT9ewx86PSrc43b-ax47l1E2UzR7Ln4j -O weights/RealESRGAN_x8.pth
%cd ..

In [ ]:
#@title 2- Iniciar sesión en HugginFace { display-mode: "form" }
from google.colab import files, output
from huggingface_hub import notebook_login

output.enable_custom_widget_manager()

notebook_login()

In [3]:
%%capture
#@title 3- Cargar los modelos
from diffusers import StableDiffusionPipeline
from diffusers import StableDiffusionImg2ImgPipeline
from diffusers import StableDiffusionInpaintPipeline

import os
import numpy as np
import torch
from typing import List, Optional, Union
import shutil
from io import BytesIO
from IPython.display import display
import PIL
from PIL import Image, ImageEnhance
from torch import autocast
from random import randint

device = "cuda"
model_path = "CompVis/stable-diffusion-v1-4"

pipe = StableDiffusionPipeline.from_pretrained(model_path, revision="fp16", torch_dtype=torch.float16, use_auth_token=True)
pipe = pipe.to("cuda")

pipe_img = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_path,
    revision="fp16", 
    torch_dtype=torch.float16,
    use_auth_token=True
)

pipe_img = pipe_img.to(device)

pipe_inp = StableDiffusionInpaintPipeline.from_pretrained(
    model_path,
    revision="fp16", 
    torch_dtype=torch.float16,
    use_auth_token=True
).to(device)

def dummy_checker(images, **kwargs): return images, False

pipe.safety_checker = dummy_checker
pipe_img.safety_checker = dummy_checker
pipe_inp.safety_checker = dummy_checker

# Generador de imágenes

In [ ]:
#@title Generar imágenes
prompt = "small videogame icon by shaddy safadi and ariel fain and calvin boice of a round bottle of red potion on a solid color background" #@param {type:"string"}
inference_steps = 20 #@param {type:"number"}
guidance_scale = 10 #@param {type:"number"}
height=512 #@param {type:"number"}
width=512 #@param {type:"number"}
random_seed = True #@param {type:"boolean"}


seed = 93916128 #@param {type:"number"}

if random_seed:
  seed = randint(0, 100000000)

print("semilla: ", seed)
generator = torch.Generator("cuda").manual_seed(seed)

with autocast("cuda"):
  image = pipe(prompt, num_inference_steps=inference_steps, generator=generator, height=height, width=width, guidance_scale=guidance_scale).images[0]
image

# Imagen a imagen

In [ ]:
#@title 1- Seleccionar imagen inicial

upload_folder = 'inputs'
result_folder = 'results'

os.makedirs(upload_folder, exist_ok=True)
os.makedirs(result_folder, exist_ok=True)

# upload files
uploaded = files.upload()

image_name = ""

for k in uploaded.keys():
  image_name = k

for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)

img = Image.open('inputs/' + image_name)
  
width = img.width
height = img.height

size = (width, height)

image = Image.open(BytesIO(uploaded[image_name])).resize(size)
backup_img = image
generated_img = image
display(image)

In [ ]:
#@title 2- Generar la nueva imagen

prompt = "videogame art by shaddy safadi and ariel fain and calvin boice of a round bottle of blue potion on a solid color background" #@param {type:"string"}
inference_steps = 20 #@param {type:"number"}
guidance_scale = 10 #@param {type:"number"}
strength = 0.8 #@param {type:"number"}
random_seed = True #@param {type:"boolean"}
iterate_over_output_image = False #@param {type:"boolean"}

seed = 35065881 #@param {type:"number"}

if random_seed:
  seed = randint(0, 100000000)

print("semilla: ", seed)

generator2 = torch.Generator(device=device).manual_seed(seed)

if iterate_over_output_image:
  image = generated_img
else:
  image = backup_img

with autocast("cuda"):
    generated_img = pipe_img(prompt=prompt, num_inference_steps=inference_steps, init_image=image, strength=strength, guidance_scale=guidance_scale, generator=generator2).images[0]


generated_img

# Inpainting

In [ ]:
#@title 1- Seleccionar imagen inicial

upload_folder = 'inputs'
result_folder = 'results'

os.makedirs(upload_folder, exist_ok=True)
os.makedirs(result_folder, exist_ok=True)

# upload files
uploaded = files.upload()

image_name = ""

for k in uploaded.keys():
  image_name = k

for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)

img = Image.open('inputs/' + image_name)
  
width = img.width
height = img.height

size = (width, height)

image_a = Image.open(BytesIO(uploaded[image_name])).resize(size)
display(image_a)

In [ ]:
#@title 2- Seleccionar máscara

upload_folder = 'inputs'
result_folder = 'results'

os.makedirs(upload_folder, exist_ok=True)
os.makedirs(result_folder, exist_ok=True)

# upload files
uploaded = files.upload()

image_name = ""

for k in uploaded.keys():
  image_name = k

for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)

img = Image.open('inputs/' + image_name)
  
width = img.width
height = img.height

size = (width, height)

image_b = Image.open(BytesIO(uploaded[image_name])).resize(size)
display(image_b)

In [ ]:
#@title 3- Generar imagen

def inpaint(p, init_image, mask_image=None, strength=0.75, guidance_scale=7.5, generator=None, num_samples=1, n_iter=1):
    all_images = []
    for _ in range(n_iter):
        with autocast("cuda"):
            images = pipe_inp(
                prompt=[p] * num_samples,
                init_image=init_image,
                mask_image=mask_image,
                strength=strength,
                guidance_scale=guidance_scale,
                generator=generator,
            ).images
        all_images.extend(images)
    return all_images


prompt = "a cat sitting on a bench" #@param {type:"string"}
strength=0.75 #@param {type:"number"}
guidance_scale=7.5 #@param {type:"number"}
num_samples = 3 #@param {type:"number"}
n_iter = 1 #@param {type:"number"}
random_seed = True #@param {type:"boolean"}


seed = 0 #@param {type:"number"}

if random_seed:
  seed = randint(0, 100000000)

generator = torch.Generator(device="cuda").manual_seed(seed) # change the seed to get different results
all_images = inpaint(prompt, image_a, image_b, strength=strength, guidance_scale=guidance_scale, generator=generator, num_samples=num_samples, n_iter=n_iter)

display(all_images[0])

# Aumentar resolución

In [ ]:
#@title 1- Seleccionar imagen
%cd Real-ESRGAN
upload_folder = 'inputs'
result_folder = 'results'

os.makedirs(upload_folder, exist_ok=True)
os.makedirs(result_folder, exist_ok=True)

# upload files
uploaded = files.upload()

img_name = ""

for k in uploaded.keys():
  img_name = k

for filename in uploaded.keys():
  dst_path = os.path.join(upload_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)

%cd ..

In [ ]:
#@title 2- Aumentar resolución

multiplier = "4" #@param ["2", "4", "8"] {allow-input: false}
int_multiplier = int(multiplier)

%cd Real-ESRGAN
from realesrgan import RealESRGAN
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

model = RealESRGAN(device, scale=int_multiplier)
weight_path = 'weights/RealESRGAN_x%s.pth' %multiplier
model.load_weights(weight_path)

path_to_image = 'inputs/' + img_name
result_image_path = 'results/sr_image.png'

image = Image.open(path_to_image).convert('RGB')
sr_image = model.predict(np.array(image))

sr_image.save(result_image_path)

%cd ..

display(sr_image)